In [1]:
from pyspark.sql.session import SparkSession

In [2]:
spark=SparkSession.builder.appName("Module 6 Case Study 1").getOrCreate()

In [3]:
#new_df = df1.join(df2, on=['field_B', 'field_C', 'field_D'], how='left_outer')

In [4]:
ordersDF = spark.read.csv("/user/edureka_524533/Datasets/orders.csv",inferSchema=True,header=True)

In [5]:
from pyspark.sql import functions as F

In [6]:
#OrderIDDF = ordersDF.groupby('order_id').agg(F.count('order_id').alias('OrderCount'))

In [7]:
#OrderIDDF.count()

# Divide Order DF into three : Prior data
#                            Train data
# Test Data

In [8]:
orderPriorDF = ordersDF.where(ordersDF['eval_set']=="prior")

In [9]:
#orderPriorDF.show(5)

In [10]:
#orderPriorDF.count()

In [11]:
orderTrainDF = ordersDF.where(ordersDF['eval_set']=="train")

In [12]:
#orderTrainDF.show(5)

In [13]:
#orderTrainDF.count()

In [14]:
#ordersDF.groupby('eval_set').agg(F.count('eval_set').alias('OrderCount')).show()

In [15]:
orderTestDF = ordersDF.where(ordersDF['eval_set']=="test")

In [16]:
#orderTestDF.show(5)

In [17]:
#orderTestDF.count()

# Now Join each individual Order DF with its counter orders_product*.csv DF

In [18]:
priorDF=spark.read.csv("/user/edureka_524533/Datasets/order_products__prior.csv",inferSchema=True,header=True)

In [19]:
#priorDF.count()

In [20]:
#priorDF.where(priorDF['order_id']==2774568).show()

In [21]:
#orderPriorDF=ordersDF.join(priorDF,(ordersDF['order_id']==priorDF['order_id'])&(ordersDF['eval_set']=='prior'),how='left_outer')

In [22]:
orderPriorJoinDF = orderPriorDF.join(priorDF,on=['order_id'],how='left_outer')

In [23]:
#orderPriorJoinDF.show(5)

In [24]:
#orderPriorJoinDF.count()

In [25]:
trainDF = spark.read.csv("/user/edureka_524533/Datasets/order_products__train.csv",inferSchema=True,header=True)

In [26]:
#trainDF.show(5)

In [27]:
#trainDF.count()

In [28]:
#trainDF.where(trainDF['order_id']==2774568).show()

In [29]:
#orderDF2=ordersDF.join(trainDF,(ordersDF['order_id']==trainDF['order_id'])&(ordersDF['eval_set']=='train'),how='left_outer')

In [30]:
orderTrainJoinDF = orderTrainDF.join(trainDF,on=['order_id'],how='left_outer')

In [31]:
#orderTrainJoinDF.show(5)

In [32]:
#orderTrainJoinDF.count()

In [33]:
#We do not have the test product details, so either we can simply ignore them or go ahead to create a join with nulls
orderTestJoinDF = orderTestDF.join(trainDF,on=['order_id'],how='left_outer')

In [34]:
#orderTestJoinDF.show(5)

In [35]:
#orderTestJoinDF.count()

# Now Join all the three OrderDFs: orderPriorJoin, orderTrainJoin, orderTestJoin

In [36]:
Orders1 = orderPriorJoinDF.unionAll(orderTrainJoinDF)

In [37]:
#Orders1.count()

In [38]:
ordersAllDF=Orders1.unionAll(orderTestJoinDF)

In [39]:
#ordersAllDF.count()

# Now we need tp Join this DF with Products.csv

In [40]:
prodDF=spark.read.csv("/user/edureka_524533/Datasets/products.csv",inferSchema=True,header=True)

In [41]:
#prodDF.show(5)

In [42]:
orderProductAllDF = ordersAllDF.join(prodDF,on='product_id',how='left_outer')

In [43]:
#orderProductAllDF.show(5)

In [44]:
#orderProductAllDF.count()

# Now Join the aisle details using aisles.csv

In [45]:
aisleDF=spark.read.csv("/user/edureka_524533/Datasets/aisles.csv",inferSchema=True,header=True)

In [46]:
#aisleDF.show(5)

In [47]:
orderProductAisleAllDF = orderProductAllDF.join(aisleDF,on='aisle_id',how='left_outer')

In [48]:
#orderProductAisleAllDF.show(5)

In [49]:
#orderProductAisleAllDF.count()

# Now join the department details as well using departments.csv

In [50]:
deptDF=spark.read.csv("/user/edureka_524533/Datasets/departments.csv",inferSchema=True,header=True)

In [51]:
#deptDF.show(5)

In [52]:
OrderProductAisleDepartAllDF = orderProductAisleAllDF.join(deptDF,on='department_id',how='left_outer')

In [53]:
OrderProductAisleDepartAllDF.show(5)

+-------------+--------+----------+--------+-------+--------+------------+---------+-----------------+----------------------+-----------------+---------+--------------------+--------------------+---------------+
|department_id|aisle_id|product_id|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|add_to_cart_order|reordered|        product_name|               aisle|     department|
+-------------+--------+----------+--------+-------+--------+------------+---------+-----------------+----------------------+-----------------+---------+--------------------+--------------------+---------------+
|            9|      63|     38650|     148|  41523|   prior|          27|        2|               17|                   5.0|                1|        0| Organic Red Lentils|grains rice dried...|dry goods pasta|
|           16|      91|     25659|     148|  41523|   prior|          27|        2|               17|                   5.0|                2|        0

In [54]:
#OrderProductAisleDepartAllDF.count()

In [55]:
#OrderProductAisleDepartAllDF.printSchema()

# Check missing data

In [56]:
missingDataDF = OrderProductAisleDepartAllDF.filter(OrderProductAisleDepartAllDF['department_id'].isNull()|
                                                   OrderProductAisleDepartAllDF['aisle_id'].isNull()|
                                                   OrderProductAisleDepartAllDF['product_id'].isNull()|
                                                   OrderProductAisleDepartAllDF['order_id'].isNull()|
                                                   OrderProductAisleDepartAllDF['user_id'].isNull()|
                                                   OrderProductAisleDepartAllDF['eval_set'].isNull()|
                                                   OrderProductAisleDepartAllDF['order_number'].isNull()|
                                                   OrderProductAisleDepartAllDF['order_dow'].isNull()|
                                                   OrderProductAisleDepartAllDF['order_hour_of_day'].isNull()|
                                                   OrderProductAisleDepartAllDF['days_since_prior_order'].isNull()|
                                                   OrderProductAisleDepartAllDF['add_to_cart_order'].isNull()|
                                                   OrderProductAisleDepartAllDF['reordered'].isNull()|
                                                   OrderProductAisleDepartAllDF['product_name'].isNull()|
                                                   OrderProductAisleDepartAllDF['aisle'].isNull()|
                                                   OrderProductAisleDepartAllDF['department'].isNull())

In [57]:
missingDataDF.count()

2153071

In [58]:
missingDataDF.show(5)

+-------------+--------+----------+--------+-------+--------+------------+---------+-----------------+----------------------+-----------------+---------+--------------------+--------------------+----------+
|department_id|aisle_id|product_id|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|add_to_cart_order|reordered|        product_name|               aisle|department|
+-------------+--------+----------+--------+-------+--------+------------+---------+-----------------+----------------------+-----------------+---------+--------------------+--------------------+----------+
|           16|     120|     18523|    2366| 160475|   prior|           1|        6|               10|                  null|                1|        0|Total 2% All Natu...|              yogurt|dairy eggs|
|            4|      24|     16797|    2366| 160475|   prior|           1|        6|               10|                  null|                2|        0|        Strawberrie

# List the most ordered products (top 10)

In [59]:
gpByProductDF = OrderProductAisleDepartAllDF.groupby('product_name').count()

In [60]:
#gpByProductDF.show(5)

+--------------------+-----+
|        product_name|count|
+--------------------+-----+
|Fresh Marinara Sauce|  483|
|Organic Raw Multi...| 6837|
|Butter Masala Mil...|  578|
|Roasted Garlic & ...| 1066|
|Organic Lentil Beans| 3430|
+--------------------+-----+
only showing top 5 rows



In [61]:
from pyspark.sql.functions import *

In [62]:
#gpByProductDF.orderBy(desc('count')).show(10)

+--------------------+------+
|        product_name| count|
+--------------------+------+
|              Banana|491291|
|Bag of Organic Ba...|394930|
|Organic Strawberries|275577|
|Organic Baby Spinach|251705|
|Organic Hass Avocado|220877|
|     Organic Avocado|184224|
|         Large Lemon|160792|
|        Strawberries|149445|
|               Limes|146660|
|  Organic Whole Milk|142813|
+--------------------+------+
only showing top 10 rows



In [63]:
Top10OrderedProducts = gpByProductDF.orderBy(col('count').desc()).limit(10)

In [64]:
Top10OrderedProducts.show()

+--------------------+------+
|        product_name| count|
+--------------------+------+
|              Banana|491291|
|Bag of Organic Ba...|394930|
|Organic Strawberries|275577|
|Organic Baby Spinach|251705|
|Organic Hass Avocado|220877|
|     Organic Avocado|184224|
|         Large Lemon|160792|
|        Strawberries|149445|
|               Limes|146660|
|  Organic Whole Milk|142813|
+--------------------+------+



# Do people usually reorder the same previous ordered products?

In [65]:
#OrderProductAisleDepartAllDF.printSchema()

root
 |-- department_id: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- eval_set: string (nullable = true)
 |-- order_number: integer (nullable = true)
 |-- order_dow: integer (nullable = true)
 |-- order_hour_of_day: integer (nullable = true)
 |-- days_since_prior_order: double (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle: string (nullable = true)
 |-- department: string (nullable = true)



In [66]:
reorderedCount = OrderProductAisleDepartAllDF.where(OrderProductAisleDepartAllDF['reordered']==1).count()

In [67]:
#reorderedCount

19955360

In [68]:
notReorderedCount = OrderProductAisleDepartAllDF.where(OrderProductAisleDepartAllDF['reordered']==0).count()

In [69]:
#notReorderedCount

13863746

In [70]:
from __future__ import division

In [71]:
# We are ignoring the 75000 test records for which we do not have the order_product__test.csv data
Total = reorderedCount + notReorderedCount

In [72]:
#Total

33819106

In [73]:
reorderPercentile = (reorderedCount/Total)*100

In [74]:
reorderPercentile

59.00617242809434

# List most reordered products

In [75]:
reorderedProdDF = OrderProductAisleDepartAllDF.where(OrderProductAisleDepartAllDF['reordered']==1)

In [76]:
#reorderedProdDF.show(5)

In [77]:
reorderedGpByProdDF = reorderedProdDF.groupby('product_name').count()

In [78]:
#reorderedGpByProdDF.show(5)

In [79]:
#reorderedGpByProdDF.orderBy(desc('count')).show(10)

+--------------------+------+
|        product_name| count|
+--------------------+------+
|              Banana|415166|
|Bag of Organic Ba...|329275|
|Organic Strawberries|214448|
|Organic Baby Spinach|194939|
|Organic Hass Avocado|176173|
|     Organic Avocado|140270|
|  Organic Whole Milk|118684|
|         Large Lemon|112178|
| Organic Raspberries|109688|
|        Strawberries|104588|
+--------------------+------+
only showing top 10 rows



In [127]:
Top10ReorderedProducts = reorderedGpByProdDF.orderBy(col('count').desc()).limit(10)

In [128]:
Top10ReorderedProducts.show()

+--------------------+------+
|        product_name| count|
+--------------------+------+
|              Banana|415166|
|Bag of Organic Ba...|329275|
|Organic Strawberries|214448|
|Organic Baby Spinach|194939|
|Organic Hass Avocado|176173|
|     Organic Avocado|140270|
|  Organic Whole Milk|118684|
|         Large Lemon|112178|
| Organic Raspberries|109688|
|        Strawberries|104588|
+--------------------+------+



# Most important aisle (by number of products)

In [80]:
#prodDF.show(5)

+----------+--------------------+--------+-------------+
|product_id|        product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
|         1|Chocolate Sandwic...|      61|           19|
|         2|    All-Seasons Salt|     104|           13|
|         3|Robust Golden Uns...|      94|            7|
|         4|Smart Ones Classi...|      38|            1|
|         5|Green Chile Anyti...|       5|           13|
+----------+--------------------+--------+-------------+
only showing top 5 rows



In [81]:
totProdPerAisle = prodDF.groupby('aisle_id').count()

In [82]:
#totProdPerAisle.show(5)

+--------+-----+
|aisle_id|count|
+--------+-----+
|     125|   69|
|       7|  100|
|      51|  264|
|     124|  195|
|      15|   80|
+--------+-----+
only showing top 5 rows



In [83]:
#totProdPerAisle.count()

135

In [84]:
#totProdPerAisleDescDF = totProdPerAisle.orderBy(desc('count'))

In [85]:
#totProdPerAisleDescDF.show(10)

In [86]:
val = totProdPerAisle.agg({'count':'max'}).collect()[0][0]

In [87]:
#val

1258

In [88]:
totProdPerAisle.createOrReplaceTempView("AisleCountTab")

In [89]:
aisle_id = spark.sql("Select * from AisleCountTab where count = {0}".format(val)).collect()[0][0]

In [90]:
#aisle_id

u'100'

In [91]:
mostImpAisle = aisleDF.where(aisleDF['aisle_id']==aisle_id)

In [92]:
mostImpAisle.show()

+--------+-------+
|aisle_id|  aisle|
+--------+-------+
|     100|missing|
+--------+-------+



In [93]:
#Looking for next most important aisle as 100 is missing from aisle.csv

In [94]:
#totProdPerAisle=totProdPerAisle.drop(totProdPerAisle['count']==aisle_id)

In [95]:
#totProdPerAisle.count()

In [96]:
nextTotProdPerAisleDF = totProdPerAisle.where(totProdPerAisle['aisle_id']!=aisle_id)

In [97]:
#nextTotProdPerAisleDF.count()

134

In [98]:
nextval = nextTotProdPerAisleDF.agg({'count':'max'}).collect()[0][0]

In [99]:
#nextval

1246

In [100]:
nextTotProdPerAisleDF.createOrReplaceTempView("NextAisleCountTab")

In [101]:
aisle_id = spark.sql("Select * from NextAisleCountTab where count = {0}".format(nextval)).collect()[0][0]

In [102]:
#aisle_id

u'45'

In [103]:
nextMostImpAisle = aisleDF.where(aisleDF['aisle_id']==aisle_id)

In [104]:
nextMostImpAisle.show()    #Most important Aisle

+--------+---------------+
|aisle_id|          aisle|
+--------+---------------+
|      45|candy chocolate|
+--------+---------------+



# Most important department (by number of products)

In [105]:
totProdPerDeptDF = prodDF.groupby('department_id').count()

In [106]:
#totProdPerDeptDF.show()

In [107]:
maxCount = totProdPerDeptDF.agg({'count':'max'}).collect()[0][0]

In [108]:
#maxCount

6563

In [109]:
totProdPerDeptDF.createOrReplaceTempView("DeptCountTab")

In [110]:
dept_id = spark.sql("Select * from DeptCountTab where count = {0}".format(maxCount)).collect()[0][0]

In [111]:
#dept_id

u'11'

In [112]:
mostImpDept = deptDF.where(deptDF['department_id']==dept_id)

In [113]:
mostImpDept.show()

+-------------+-------------+
|department_id|   department|
+-------------+-------------+
|           11|personal care|
+-------------+-------------+



# Get the Top 10 departments

In [114]:
#OrderProductAisleDepartAllDF.show(5)

In [115]:
totProdOrderedPerDeptDF = OrderProductAisleDepartAllDF.groupby('department').count()

In [116]:
#totProdOrderedPerDeptDF.show(5)

In [117]:
#totProdOrderedPerDeptDF.orderBy(desc('count')).show(10)

In [118]:
Top10Departments = totProdOrderedPerDeptDF.orderBy(col('count').desc()).limit(10)

In [119]:
Top10Departments.show()

+---------------+-------+
|     department|  count|
+---------------+-------+
|        produce|9888378|
|     dairy eggs|5631067|
|         snacks|3006412|
|      beverages|2804175|
|         frozen|2336858|
|         pantry|1956819|
|         bakery|1225181|
|   canned goods|1114857|
|           deli|1095540|
|dry goods pasta| 905340|
+---------------+-------+



# List top 10 products ordered in the morning (6 AM to 11 AM)

In [120]:
morningOrdersDF=OrderProductAisleDepartAllDF.where((OrderProductAisleDepartAllDF['order_hour_of_day']>=6)&(OrderProductAisleDepartAllDF['order_hour_of_day']<=11))

In [121]:
#morningOrdersDF.show(5)

In [122]:
totProdOrderedMornDF = morningOrdersDF.groupby('product_name').count()

In [123]:
#totProdOrderedMornDF.show()

In [124]:
#totProdOrderedMornDF.orderBy(desc('count')).show(10)

In [125]:
Top10MornProducts = totProdOrderedMornDF.orderBy(col('count').desc()).limit(10)

In [126]:
Top10MornProducts.show()

+--------------------+------+
|        product_name| count|
+--------------------+------+
|              Banana|169965|
|Bag of Organic Ba...|135417|
|Organic Strawberries| 92499|
|Organic Baby Spinach| 82578|
|Organic Hass Avocado| 72545|
|     Organic Avocado| 59603|
|         Large Lemon| 53479|
|        Strawberries| 52155|
| Organic Raspberries| 49751|
|  Organic Whole Milk| 49747|
+--------------------+------+

